In [177]:
import pandas as pd
import numpy as np
import miceforest as mf
from math import nan
# Note: properly installing lightgbm allows you to run miceforest. If you have an M1 mac, please see:
# https://towardsdatascience.com/install-xgboost-and-lightgbm-on-apple-m1-macs-cb75180a2dda

In [178]:
student_survey = pd.read_csv('data/36423-0002-Data.tsv',sep='\t')

### Math Enrollment

S2MSPR12: S2 D13 Teenager taking math class(es) in spring 2012
[Are you currently/Were you] taking a math course [during the spring term of 2012?]
1=Yes
0=No

S1MFALL09: S1 C03 9th grader is taking a math course in the fall 2009 term

In [179]:
student_survey = student_survey[(student_survey['S2MSPR12'] == 1) & (student_survey['S1MFALL09'] == 1)]
len(student_survey)

14575

### Sample size rounded to the nearest ten in accordance to NCES regulation.
Throughout their paper.

### Low teacher support 

S2MTCHTREAT: S2 D18A Teen's spring 2012 math teacher treats some kids better than others
How much do you agree or disagree with the following statements about your teacher for [math course
title]? Remember, none of your teachers or your principal will see any of the answers you provide.
Your teacher...
[treats/treated] some kids better than other kids.
1=Strongly agree
2=Agree

S2MTCHINTRST: S2 D18B Teen's spring 2012 math teacher makes math interesting
How much do you agree or disagree with the following statements about your teacher for [math course
title]? Remember, none of your teachers or your principal will see any of the answers you provide.
Your teacher...
[makes/made] math interesting.
1=Strongly agree
2=Agree
3=Disagree
4=Strongly disagree


S2MTCHEASY: S2 D18C Teen's spring 2012 math teacher makes math easy to understand
How much do you agree or disagree with the following statements about your teacher for [math course
title]? Remember, none of your teachers or your principal will see any of the answers you provide.
Your teacher...
[makes/made] math easy to understand.
1=Strongly agree
2=Agree
3=Disagree
4=Strongly disagree

S2MTCHTHINK: S2 D18D Teen's spring 2012 math teacher wants students to think, not
memorize
How much do you agree or disagree with the following statements about your teacher for [math course
title]? Remember, none of your teachers or your principal will see any of the answers you provide.
Your teacher...
[wants/wanted] students to think, not just memorize things.
1=Strongly agree
2=Agree
3=Disagree
4=Strongly disagree

S2MTCHGIVEUP: S2 D18E Teen's spring 2012 math teacher doesn't let students give up
title]? Remember, none of your teachers or your principal will see any of the answers you provide.
Your teacher...
[doesn't/didn't] let people give up when the work [gets/got] hard.
1=Strongly agree
2=Agree
3=Disagree
4=Strongly disagree

### Ability self-concepts

S1MTESTS: S1 C08A 9th grader confident can do excellent job on fall 2009 math tests
How much do you agree or disagree with the following statements about your [fall 2009 math] course?
You are confident that you can do an excellent job on tests in this course
Strongly agree
Agree

S1MTEXTBOOK: S1 C08B 9th grader certain can understand fall 2009 math textbook
How much do you agree or disagree with the following statements about your [fall 2009 math] course?
You are certain that you can understand the most difficult material presented in the textbook used
in this course
Strongly agree
Agree
Disagree
Strongly disagree

S1MSKILLS: S1 C08C 9th grader certain can master skills in fall 2009 math course
How much do you agree or disagree with the following statements about your [fall 2009 math] course?
You are certain that you can master the skills being taught in this course
Strongly agree
Agree
Disagree
Strongly disagree

S1MASSEXCL: S1 C08D 9th grader confident can do excellent job on fall 2009 math
assignments
How much do you agree or disagree with the following statements about your [fall 2009 math] course?
You are confident that you can do an excellent job on assignments in this course
Strongly agree
Agree
Disagree
Strongly disagree

### Parental support

P1MUSEUM: P1 E07A Went to science or engineering museum with 9th grader in last year
P2MUSEUM: P2 B10A Visited science-related destination together in last year
Value Label Unweighted
Frequency
%
0 No 7195 30.6 %
1 Yes 8253 35.1 %
Missing Data
-9 Missing 1340 5.7 %
-8 Unit non-response 6715 28.6 %

Value Label Unweighted
Frequency
%
0 No 4837 20.6 %
1 Yes 3248 13.8 %
Missing Data
-9 Missing 63 0.3 %
-8 Unit non-response 2603 11.1 %
-6 Component not applicable 12279 52.2 %
-4 Item not administered: abbreviated interview 473 2.0 %

P1COMPUTER: P1 E07B Worked or played on computer with 9th grader in last year
P2COMPUTER: P2 B10B Worked or played on computer with teenager in last year

P1FIXED: P1 E07C Built or fixed something with 9th grader in last year
P2FIXED: P2 B10C Built or fixed something with teenager in last year

P1LIBRARY: P1 E07G Visited a library with 9th grader in last year
P2LIBRARY: P2 B10F Visited a library with teenager in last year

P1STEMDISC: P1 E07F Discussed STEM program or article with 9th grader in last year
P2STEMDISC: P2 B10E Discussed STEM program or article with teenager in last year

### Math Acheivement Score

X2TXMSCR: X2 Mathematics IRT-estimated number right score (of ## first follow-up items)
Based upon 20,594 valid cases out of 23,503 total cases.
• Mean: 67.2219
• Minimum: 25.0057
• Maximum: 115.1000
• Standard Deviation: 19.2183

X2X1TXMSCR: X2 Mathematics IRT-estimated number right score at time of base year (of 118
first follow-up items)

X3THIMATH9: X3 Highest level mathematics course taken - ninth grade
13 AP/IB Calculus 0 0.0 %
Missing Data
-9 Missing 770 3.3 %
-8 Unit non-response

In [180]:
all_columns = [
    'S2MSPR12',
    'S1MFALL09',
    'S2MTCHTREAT', 
    'S2MTCHINTRST',
    'S2MTCHEASY',
    'S2MTCHTHINK',
    'S2MTCHGIVEUP',
    'S1MTESTS',
    'S1MTEXTBOOK',
    'S1MSKILLS',
    'S1MASSEXCL',
    'P1MUSEUM',
    'P2MUSEUM',
    'P1COMPUTER',
    'P2COMPUTER',
    'P1FIXED',
    'P2FIXED',
    'P1LIBRARY',
    'P2LIBRARY',
    'P1STEMDISC',
    'P2STEMDISC',
    'X2TXMSCR',
    'X2X1TXMSCR',
    'X1SEX',
    'X1RACE',
    'X1SES_U',
    'X3THIMATH9',
    'X1TXMSCR',
    'W1PARENT'
]

low_teacher_support = [
    ('S2MTCHTREAT','treats some kids better'),
    ('S2MTCHINTRST','makes math interesting'),
    ('S2MTCHEASY','makes math easy to understand'),
    ('S2MTCHTHINK','wants students to think'),
    ('S2MTCHGIVEUP','doesnt let students give up')
]

ability_self_concept = [
    ('S1MTESTS','confident can do excellent job on test'),
    ('S1MTEXTBOOK','certain can understand math textbook'),
    ('S1MSKILLS','certain can master math skills'),
    ('S1MASSEXCL','confident can do excellent job on assignments')
]

parental_support = [
    ('P1MUSEUM','went to science or engineering museum'),
    ('P2MUSEUM','went to science or engineering museum'),
    ('P1COMPUTER','worked or played on computer'),
    ('P2COMPUTER','worked or played on computer'),
    ('P1FIXED','built or fixed something'),
    ('P2FIXED','built or fixed something'),
    ('P1LIBRARY','visited a library'),
    ('P2LIBRARY','visited a library'),
    ('P1STEMDISC','discussed STEM program or article'),
    ('P2STEMDISC','discussed STEM program or article'),]

covariates = [
    'sex',
    'race',
    'SES',
    'base_year_score',
    'highest_level_math'
]

math_acheivement_score = [('X2TXMSCR', 'score')]

highest_level_math = [('X3THIMATH9', 'level')]

base_year_score = [('X1TXMSCR', 'base_score')]

Students’ demographic information including their gender, race/ethnicity, 9 grade math achievement using the IRT-estimated
score (i.e., a criterion-referenced measure of achievement on algebraic reasoning assessment which was similarly constructed and administered as the assessment in 11th grade), and socioeconomic status (i.e., a composite measure of parents’ education, occupation, and family income) collected in 9th grade were included in the analyses as covariates. Also, students’ highest-level math course taken in 9th grade (1 = Basic math, 13 = AP/IB calculus) from the high school transcript was included.

In [181]:
student_survey = student_survey[all_columns]

student_survey['sex'] = student_survey['X1SEX']
student_survey.loc[student_survey["sex"] == 1, "sex"] = 0
student_survey.loc[student_survey["sex"] == 2, "sex"] = 1
student_survey['sex'] = student_survey['sex'].astype(np.int8)

student_survey['race'] = student_survey['X1RACE']
student_survey.loc[student_survey["race"] == 8, "race"] = 10
student_survey.loc[student_survey["race"] == 3, "race"] = 11
student_survey.loc[student_survey["race"] == 4, "race"] = 12
student_survey.loc[student_survey["race"] == 5, "race"] = 12
student_survey.loc[student_survey["race"] == 2, "race"] = 13
student_survey.loc[student_survey["race"] == 6, "race"] = 14
student_survey.loc[student_survey["race"] == 7, "race"] = 15
student_survey.loc[student_survey["race"] == 1, "race"] = 15
student_survey['race'] = student_survey['race'] - 10
student_survey['race'] = student_survey['race'].astype(np.int8)

student_survey['SES'] = student_survey['X1SES_U']
# pd.qcut(student_survey['X1SES_U'],4,labels=np.arange(4) + 1)
# student_survey['SES'] = student_survey['SES'].astype(np.int8)

In [184]:
weights = student_survey['W1PARENT']
student_survey.drop(['W1PARENT'], axis=1)

,S2MSPR12,S1MFALL09,S2MTCHTREAT,S2MTCHINTRST,S2MTCHEASY,S2MTCHTHINK,S2MTCHGIVEUP,S1MTESTS,S1MTEXTBOOK,S1MSKILLS,...,X2TXMSCR,X2X1TXMSCR,X1SEX,X1RACE,X1SES_U,X3THIMATH9,X1TXMSCR,sex,race,SES
0,1,1,2,2,2,2,3,1,2,2,...,99.1403,69.4994,1,8,1.6907,6,50.4919,0,0,1.6907
1,1,1,3,3,3,2,2,2,2,2,...,72.4904,49.4710,2,8,-0.3923,2,35.8045,1,0,-0.3923
2,1,1,3,1,1,2,1,1,3,2,...,75.4243,77.3584,2,3,1.1271,5,56.0477,1,1,1.1271
6,1,1,2,4,4,2,3,2,2,2,...,60.0290,52.7841,2,8,-0.4774,3,38.4063,1,0,-0.4774
9,1,1,2,3,2,2,3,1,1,1,...,86.8286,76.6330,2,8,0.1081,4,55.5463,1,0,0.1081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23497,1,1,3,1,1,1,1,2,2,2,...,85.8476,70.7533,1,8,1.2713,-9,51.3889,0,0,1.2713
23499,1,1,3,3,2,1,1,2,1,2,...,60.8373,46.4682,2,5,-1.3350,3,33.3823,1,2,-1.3350
23500,1,1,1,1,1,1,1,2,3,2,...,60.9564,53.2958,2,8,-0.0031,4,38.8000,1,0,-0.0031
23501,1,1,2,4,4,3,4,2,2,2,...,65.1187,72.6301,1,8,0.7236,4,52.7281,0,0,0.7236


In [185]:
student_survey = student_survey.apply(pd.to_numeric, errors = 'coerce')
student_survey

,S2MSPR12,S1MFALL09,S2MTCHTREAT,S2MTCHINTRST,S2MTCHEASY,S2MTCHTHINK,S2MTCHGIVEUP,S1MTESTS,S1MTEXTBOOK,S1MSKILLS,...,X2X1TXMSCR,X1SEX,X1RACE,X1SES_U,X3THIMATH9,X1TXMSCR,W1PARENT,sex,race,SES
0,1,1,2,2,2,2,3,1,2,2,...,69.4994,1,8,1.6907,6,50.4919,470.250141,0,0,1.6907
1,1,1,3,3,3,2,2,2,2,2,...,49.4710,2,8,-0.3923,2,35.8045,224.455466,1,0,-0.3923
2,1,1,3,1,1,2,1,1,3,2,...,77.3584,2,3,1.1271,5,56.0477,185.301339,1,1,1.1271
6,1,1,2,4,4,2,3,2,2,2,...,52.7841,2,8,-0.4774,3,38.4063,379.440827,1,0,-0.4774
9,1,1,2,3,2,2,3,1,1,1,...,76.6330,2,8,0.1081,4,55.5463,242.626125,1,0,0.1081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23497,1,1,3,1,1,1,1,2,2,2,...,70.7533,1,8,1.2713,-9,51.3889,73.287998,0,0,1.2713
23499,1,1,3,3,2,1,1,2,1,2,...,46.4682,2,5,-1.3350,3,33.3823,10.120169,1,2,-1.3350
23500,1,1,1,1,1,1,1,2,3,2,...,53.2958,2,8,-0.0031,4,38.8000,98.823515,1,0,-0.0031
23501,1,1,2,4,4,3,4,2,2,2,...,72.6301,1,8,0.7236,4,52.7281,262.402860,0,0,0.7236


### Imputation
The authors specify that they use ``multiple imputation'' procedures to impute missing data. Because they do not specify which procedure they use (beyond that they use the STATA package), we do best practice work here (in python) and use an MI library based on LightGBM and the MICE algorithm.

In [186]:
temp = student_survey.loc[:, student_survey.columns != 'SES']
temp[temp < 0] = nan
student_survey.loc[:, student_survey.columns != 'SES'] = temp

/var/folders/qj/gh_j11514m37mqtfrlr885k40000gn/T/ipykernel_92360/313373003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp < 0] = nan
/var/folders/qj/gh_j11514m37mqtfrlr885k40000gn/T/ipykernel_92360/313373003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[temp < 0] = nan


In [187]:
student_survey

,S2MSPR12,S1MFALL09,S2MTCHTREAT,S2MTCHINTRST,S2MTCHEASY,S2MTCHTHINK,S2MTCHGIVEUP,S1MTESTS,S1MTEXTBOOK,S1MSKILLS,...,X2X1TXMSCR,X1SEX,X1RACE,X1SES_U,X3THIMATH9,X1TXMSCR,W1PARENT,sex,race,SES
0,1,1,2.0,2.0,2.0,2.0,3.0,1.0,2.0,2.0,...,69.4994,1,8,1.6907,6.0,50.4919,470.250141,0,0,1.6907
1,1,1,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,...,49.4710,2,8,NaN,2.0,35.8045,224.455466,1,0,-0.3923
2,1,1,3.0,1.0,1.0,2.0,1.0,1.0,3.0,2.0,...,77.3584,2,3,1.1271,5.0,56.0477,185.301339,1,1,1.1271
6,1,1,2.0,4.0,4.0,2.0,3.0,2.0,2.0,2.0,...,52.7841,2,8,NaN,3.0,38.4063,379.440827,1,0,-0.4774
9,1,1,2.0,3.0,2.0,2.0,3.0,1.0,1.0,1.0,...,76.6330,2,8,0.1081,4.0,55.5463,242.626125,1,0,0.1081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23497,1,1,3.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,...,70.7533,1,8,1.2713,NaN,51.3889,73.287998,0,0,1.2713
23499,1,1,3.0,3.0,2.0,1.0,1.0,2.0,1.0,2.0,...,46.4682,2,5,NaN,3.0,33.3823,10.120169,1,2,-1.3350
23500,1,1,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,...,53.2958,2,8,NaN,4.0,38.8000,98.823515,1,0,-0.0031
23501,1,1,2.0,4.0,4.0,3.0,4.0,2.0,2.0,2.0,...,72.6301,1,8,0.7236,4.0,52.7281,262.402860,0,0,0.7236


In [188]:
# Using pip
# ! pip install miceforest --no-cache-dir

In [189]:
# Create kernel. 
kds = mf.ImputationKernel(
  student_survey,
  datasets=1,
  save_all_iterations=False,
  random_state=42
)

# Run the MICE algorithm for 2 iterations
kds.mice(2)



In [190]:
completed_dataset = kds.complete_data(dataset=0, inplace=False)
student_survey.describe() - completed_dataset.describe()

,S2MSPR12,S1MFALL09,S2MTCHTREAT,S2MTCHINTRST,S2MTCHEASY,S2MTCHTHINK,S2MTCHGIVEUP,S1MTESTS,S1MTEXTBOOK,S1MSKILLS,...,X2X1TXMSCR,X1SEX,X1RACE,X1SES_U,X3THIMATH9,X1TXMSCR,W1PARENT,sex,race,SES
count,0.0,0.0,-243.000000,-243.000000,-252.000000,-257.000000,-271.000000,-67.000000,-98.000000,-114.000000,...,0.0,0.0,0.0,-6624.000000,-1057.000000,0.0,0.0,0.0,0.0,0.0
mean,0.0,0.0,0.000336,-0.001188,-0.000076,-0.001923,-0.001205,-0.000210,-0.000826,-0.000024,...,0.0,0.0,0.0,0.317936,0.002861,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,-0.000195,0.000774,-0.000029,-0.000579,0.000269,0.000669,0.000406,0.000537,...,0.0,0.0,0.0,0.011837,-0.004274,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.257250,0.000000,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.517000,0.000000,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.384050,0.000000,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [191]:
#student_survey = student_survey[(student_survey[all_columns] >= 0).all(axis=1)]
student_survey

,S2MSPR12,S1MFALL09,S2MTCHTREAT,S2MTCHINTRST,S2MTCHEASY,S2MTCHTHINK,S2MTCHGIVEUP,S1MTESTS,S1MTEXTBOOK,S1MSKILLS,...,X2X1TXMSCR,X1SEX,X1RACE,X1SES_U,X3THIMATH9,X1TXMSCR,W1PARENT,sex,race,SES
0,1,1,2.0,2.0,2.0,2.0,3.0,1.0,2.0,2.0,...,69.4994,1,8,1.6907,6.0,50.4919,470.250141,0,0,1.6907
1,1,1,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,...,49.4710,2,8,NaN,2.0,35.8045,224.455466,1,0,-0.3923
2,1,1,3.0,1.0,1.0,2.0,1.0,1.0,3.0,2.0,...,77.3584,2,3,1.1271,5.0,56.0477,185.301339,1,1,1.1271
6,1,1,2.0,4.0,4.0,2.0,3.0,2.0,2.0,2.0,...,52.7841,2,8,NaN,3.0,38.4063,379.440827,1,0,-0.4774
9,1,1,2.0,3.0,2.0,2.0,3.0,1.0,1.0,1.0,...,76.6330,2,8,0.1081,4.0,55.5463,242.626125,1,0,0.1081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23497,1,1,3.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,...,70.7533,1,8,1.2713,NaN,51.3889,73.287998,0,0,1.2713
23499,1,1,3.0,3.0,2.0,1.0,1.0,2.0,1.0,2.0,...,46.4682,2,5,NaN,3.0,33.3823,10.120169,1,2,-1.3350
23500,1,1,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,...,53.2958,2,8,NaN,4.0,38.8000,98.823515,1,0,-0.0031
23501,1,1,2.0,4.0,4.0,3.0,4.0,2.0,2.0,2.0,...,72.6301,1,8,0.7236,4.0,52.7281,262.402860,0,0,0.7236


### Combine columns into predictors

In [192]:
low = [i[0] for i in low_teacher_support]
student_df = completed_dataset[low].dropna()
teacher_var = (student_df.sum(axis=1)/len(low)).to_frame()
teacher_var.describe()
teacher_var.columns = ['teacher']
teacher_var

,teacher
0,2.2
1,2.6
2,1.6
6,3.0
9,2.4
...,...
23497,1.4
23499,2.0
23500,1.0
23501,3.4


In [193]:
# he scale was reverse-coded so that high scores signified strong math ability self-concepts (1 = Strongly disagree, 4 = Strongly agree).
reverse_code = {
    'S1MTESTS': {1: 4, 2: 3, 3: 2, 4: 1},
    'S1MTEXTBOOK': {1: 4, 2: 3, 3: 2, 4: 1},
    'S1MSKILLS': {1: 4, 2: 3, 3: 2, 4: 1},
    'S1MASSEXCL': {1: 4, 2: 3, 3: 2, 4: 1},
}

ability = [i[0] for i in ability_self_concept]
ability_df = completed_dataset[ability].dropna()
ability_df = ability_df.replace(reverse_code)
ability_var = (ability_df.sum(axis=1)/len(ability)).to_frame()
ability_var.columns = ['ability']
ability_var.describe()

,ability
count,14575.000000
mean,2.975146
std,0.643018
min,1.000000
25%,2.750000
50%,3.000000
75%,3.500000
max,4.000000


In [194]:
parent = [i[0] for i in parental_support]
parent_df = completed_dataset[parent]
parent_df

parental_var = (parent_df.sum(axis=1)/len(parent)).to_frame()
parental_var.columns = ['parents']
parental_var.describe()

,parents
count,14575.000000
mean,0.614772
std,0.235764
min,0.000000
25%,0.500000
50%,0.600000
75%,0.800000
max,1.000000


In [195]:
level = [i[0] for i in highest_level_math]
level_var = completed_dataset[level]
level_var.columns = ['base_level']
level_var.describe()

,base_level
count,14575.000000
mean,4.364871
std,1.464075
min,0.000000
25%,4.000000
50%,4.000000
75%,5.000000
max,11.000000


In [196]:
acheive = [i[0] for i in math_acheivement_score]
acheive_var = completed_dataset[acheive]
acheive_var.describe()
acheive_var['math'] = acheive_var['X2TXMSCR']
acheive_var = acheive_var.drop(['X2TXMSCR'], axis=1)
acheive_var.describe()

/var/folders/qj/gh_j11514m37mqtfrlr885k40000gn/T/ipykernel_92360/3484896284.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acheive_var['math'] = acheive_var['X2TXMSCR']


,math
count,14575.000000
mean,69.946709
std,18.729319
min,25.005700
25%,58.915250
50%,69.429700
75%,85.156150
max,115.100000


In [197]:
base = [i[0] for i in base_year_score]
base_var = completed_dataset[base]
base_var.describe()
base_var['base_math'] = base_var['X1TXMSCR']
base_var = base_var.drop(['X1TXMSCR'], axis=1)
base_var.describe()

/var/folders/qj/gh_j11514m37mqtfrlr885k40000gn/T/ipykernel_92360/3940350869.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_var['base_math'] = base_var['X1TXMSCR']


,base_math
count,14575.000000
mean,42.115971
std,11.610209
min,15.864100
25%,34.508650
50%,42.208800
75%,50.709000
max,69.931700


In [198]:
completed_dataset['SES'].describe()

count    14575.000000
mean         0.148680
std          0.789495
min         -1.906800
25%         -0.427050
50%          0.092500
75%          0.676300
max          2.978300
Name: SES, dtype: float64

In [199]:
full_df = pd.concat([acheive_var, 
                     teacher_var, 
                     ability_var, 
                     parental_var, 
                     completed_dataset['sex'], 
                     completed_dataset['SES'], 
                     base_var,
                     level_var], axis=1)

In [200]:
full_df.corr()
# YAY! NOTE: probably using the wrong SES - try and figure out which one they actually used...

,math,teacher,ability,parents,sex,SES,base_math,base_level
math,1.000000,-0.122527,0.299191,0.142562,-0.022433,0.414977,0.745042,0.358769
teacher,-0.122527,1.000000,-0.138090,-0.034284,0.031998,-0.051738,-0.086148,-0.042563
ability,0.299191,-0.138090,1.000000,0.082273,-0.105274,0.130770,0.302118,0.138293
parents,0.142562,-0.034284,0.082273,1.000000,-0.055088,0.225970,0.131080,0.102144
sex,-0.022433,0.031998,-0.105274,-0.055088,1.000000,-0.004499,-0.018885,0.022568
SES,0.414977,-0.051738,0.130770,0.225970,-0.004499,1.000000,0.408821,0.229129
base_math,0.745042,-0.086148,0.302118,0.131080,-0.018885,0.408821,1.000000,0.383010
base_level,0.358769,-0.042563,0.138293,0.102144,0.022568,0.229129,0.383010,1.000000


In [201]:
RACE_MAP = {
    0: "White",
    1: "Black",
    2: "Hispanic",
    3: "Asian",
    4: "Other",
    5: "Other"
}

SEX_MAP = {
    0: "Male",
    1: "Female"
}

In [202]:
full_df_regression = full_df.copy()
full_df_regression['race'] = completed_dataset['race']
full_df_regression = full_df_regression.replace({'sex': SEX_MAP,
                                                 'race': RACE_MAP})

In [203]:
full_df_regression

,math,teacher,ability,parents,sex,SES,base_math,base_level,race
0,99.1403,2.2,3.50,0.6,Male,1.6907,50.4919,6.0,White
1,72.4904,2.6,3.25,0.0,Female,-0.3923,35.8045,2.0,White
2,75.4243,1.6,3.25,0.8,Female,1.1271,56.0477,5.0,Black
6,60.0290,3.0,3.00,0.8,Female,-0.4774,38.4063,3.0,White
9,86.8286,2.4,4.00,0.6,Female,0.1081,55.5463,4.0,White
...,...,...,...,...,...,...,...,...,...
23497,85.8476,1.4,3.00,0.8,Male,1.2713,51.3889,5.0,White
23499,60.8373,2.0,3.50,0.4,Female,-1.3350,33.3823,3.0,Hispanic
23500,60.9564,1.0,2.75,0.6,Female,-0.0031,38.8000,4.0,White
23501,65.1187,3.4,3.00,0.7,Male,0.7236,52.7281,4.0,White


In [208]:
from statsmodels.regression.linear_model import WLS

model_lts = WLS.from_formula(
    'math ~ teacher + C(sex, Treatment(reference="Female")) + C(race, Treatment(reference="White")) + SES + base_math + base_level',
    data=full_df_regression,
    freq_weights=np.array(weights.array))
regression_lts = model_lts.fit(method='pinv')

/Users/lucasrosenblatt/mambaforge/envs/boost/lib/python3.8/site-packages/statsmodels/base/model.py:127: ValueWarning: unknown kwargs ['freq_weights']
  warnings.warn(msg, ValueWarning)


In [209]:
print(regression_lts.summary2())

                                 Results: Weighted least squares
Model:                        WLS                        Adj. R-squared:               0.581      
Dependent Variable:           math                       AIC:                          114118.0253
Date:                         2022-08-09 15:46           BIC:                          114193.8959
No. Observations:             14575                      Log-Likelihood:               -57049.    
Df Model:                     9                          F-statistic:                  2243.      
Df Residuals:                 14565                      Prob (F-statistic):           0.00       
R-squared:                    0.581                      Scale:                        147.11     
--------------------------------------------------------------------------------------------------
                                                   Coef.  Std.Err.    t     P>|t|   [0.025  0.975]
--------------------------------------------